In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1684424616139, experiment_id='1', last_update_time=1684424616139, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715199477344

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "James")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[14:57:33] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                                                                          
[0]	validation-rmse:8.80519                                                                                                                                                                                     
[1]	validation-rmse:7.04513                                                                                                                                                                                     
[2]	validation-rmse:6.80524                                                                                                                                                                                     
[3]	validation-rmse:6.76218                                                                                                                                         

[77]	validation-rmse:6.54563                                                                                                                                                                                    
[78]	validation-rmse:6.54440                                                                                                                                                                                    
[79]	validation-rmse:6.54372                                                                                                                                                                                    
[80]	validation-rmse:6.54156                                                                                                                                                                                    
[81]	validation-rmse:6.53974                                                                                                                                        

[155]	validation-rmse:6.46153                                                                                                                                                                                   
[156]	validation-rmse:6.46113                                                                                                                                                                                   
[157]	validation-rmse:6.45982                                                                                                                                                                                   
[158]	validation-rmse:6.45902                                                                                                                                                                                   
[159]	validation-rmse:6.45800                                                                                                                                       

[233]	validation-rmse:6.40949                                                                                                                                                                                   
[234]	validation-rmse:6.40938                                                                                                                                                                                   
[235]	validation-rmse:6.40974                                                                                                                                                                                   
[236]	validation-rmse:6.40933                                                                                                                                                                                   
[237]	validation-rmse:6.40850                                                                                                                                       

[311]	validation-rmse:6.38516                                                                                                                                                                                   
[312]	validation-rmse:6.38652                                                                                                                                                                                   
[313]	validation-rmse:6.38604                                                                                                                                                                                   
[314]	validation-rmse:6.38594                                                                                                                                                                                   
[315]	validation-rmse:6.38606                                                                                                                                       

...

[457]	validation-rmse:6.46803                                                                                                                                                                                   
[458]	validation-rmse:6.46805                                                                                                                                                                                   
[459]	validation-rmse:6.46801                                                                                                                                                                                   
[460]	validation-rmse:6.46804                                                                                                                                                                                   
[461]	validation-rmse:6.46811                                                                                                                                       

[535]	validation-rmse:6.45620                                                                                                                                                                                   
[536]	validation-rmse:6.45623                                                                                                                                                                                   
[537]	validation-rmse:6.45590                                                                                                                                                                                   
[538]	validation-rmse:6.45585                                                                                                                                                                                   
[539]	validation-rmse:6.45555                                                                                                                                       

[613]	validation-rmse:6.44423                                                                                                                                                                                   
[614]	validation-rmse:6.44396                                                                                                                                                                                   
[615]	validation-rmse:6.44371                                                                                                                                                                                   
[616]	validation-rmse:6.44332                                                                                                                                                                                   
[617]	validation-rmse:6.44384                                                                                                                                       

[691]	validation-rmse:6.43534                                                                                                                                                                                   
[692]	validation-rmse:6.43529                                                                                                                                                                                   
 78%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▋                              | 39/50 [25:39<07:14, 39.48s/trial, best loss: 6.288402599557681]


KeyboardInterrupt: 

In [14]:
# mlflow.xgboost.autolog(disable=True)
mlflow.xgboost.autolog(disable=False)

In [19]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.2798121113127615,
        'max_depth': 13,
        'min_child_weight': 1.146875313093477,
        'objective': 'reg:linear',
        'reg_alpha': 0.16063325941692372,
        'reg_lambda': 0.01711465230001588,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[16:48:00] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.28024
[1]	validation-rmse:12.90762
[2]	validation-rmse:10.64201
[3]	validation-rmse:9.16724
[4]	validation-rmse:8.22788
[5]	validation-rmse:7.63813
[6]	validation-rmse:7.26867
[7]	validation-rmse:7.03711
[8]	validation-rmse:6.89036
[9]	validation-rmse:6.79237
[10]	validation-rmse:6.72995
[11]	validation-rmse:6.68491
[12]	validation-rmse:6.65690
[13]	validation-rmse:6.63625
[14]	validation-rmse:6.62159
[15]	validation-rmse:6.61228
[16]	validation-rmse:6.60582
[17]	validation-rmse:6.59967
[18]	validation-rmse:6.59393
[19]	validation-rmse:6.58980
[20]	validation-rmse:6.58548
[21]	validation-rmse:6.58307
[22]	validation-rmse:6.58122
[23]	validation-rmse:6.57878
[24]	validation-rmse:6.57587
[25]	validation-rmse:6.57363
[26]	validation-rmse:6.57140
[27]	validation-rmse:6.56977
[28]	validation-rmse:6.56728
[29]	validation-rmse:6.56530
[30]	validation-rm

[273]	validation-rmse:6.37844
[274]	validation-rmse:6.37800
[275]	validation-rmse:6.37749
[276]	validation-rmse:6.37770
[277]	validation-rmse:6.37710
[278]	validation-rmse:6.37693
[279]	validation-rmse:6.37666
[280]	validation-rmse:6.37674
[281]	validation-rmse:6.37561
[282]	validation-rmse:6.37516
[283]	validation-rmse:6.37497
[284]	validation-rmse:6.37438
[285]	validation-rmse:6.37413
[286]	validation-rmse:6.37357
[287]	validation-rmse:6.37307
[288]	validation-rmse:6.37259
[289]	validation-rmse:6.37219
[290]	validation-rmse:6.37214
[291]	validation-rmse:6.37145
[292]	validation-rmse:6.37167
[293]	validation-rmse:6.37178
[294]	validation-rmse:6.37175
[295]	validation-rmse:6.37211
[296]	validation-rmse:6.37181
[297]	validation-rmse:6.37141
[298]	validation-rmse:6.37138
[299]	validation-rmse:6.37118
[300]	validation-rmse:6.37099
[301]	validation-rmse:6.37084
[302]	validation-rmse:6.37052
[303]	validation-rmse:6.37007
[304]	validation-rmse:6.36970
[305]	validation-rmse:6.36906
[306]	vali

[547]	validation-rmse:6.32087
[548]	validation-rmse:6.32075
[549]	validation-rmse:6.32081
[550]	validation-rmse:6.32060
[551]	validation-rmse:6.32059
[552]	validation-rmse:6.32077
[553]	validation-rmse:6.32068
[554]	validation-rmse:6.32037
[555]	validation-rmse:6.32020
[556]	validation-rmse:6.32024
[557]	validation-rmse:6.31987
[558]	validation-rmse:6.31976
[559]	validation-rmse:6.31976
[560]	validation-rmse:6.31973
[561]	validation-rmse:6.31967
[562]	validation-rmse:6.31906
[563]	validation-rmse:6.31912
[564]	validation-rmse:6.31888
[565]	validation-rmse:6.31878
[566]	validation-rmse:6.31862
[567]	validation-rmse:6.31867
[568]	validation-rmse:6.31831
[569]	validation-rmse:6.31808
[570]	validation-rmse:6.31796
[571]	validation-rmse:6.31784
[572]	validation-rmse:6.31776
[573]	validation-rmse:6.31755
[574]	validation-rmse:6.31750
[575]	validation-rmse:6.31748
[576]	validation-rmse:6.31720
[577]	validation-rmse:6.31707
[578]	validation-rmse:6.31734
[579]	validation-rmse:6.31711
[580]	vali

[821]	validation-rmse:6.29877
[822]	validation-rmse:6.29859
[823]	validation-rmse:6.29855
[824]	validation-rmse:6.29865
[825]	validation-rmse:6.29883
[826]	validation-rmse:6.29872
[827]	validation-rmse:6.29870
[828]	validation-rmse:6.29859
[829]	validation-rmse:6.29851
[830]	validation-rmse:6.29847
[831]	validation-rmse:6.29872
[832]	validation-rmse:6.29877
[833]	validation-rmse:6.29862
[834]	validation-rmse:6.29841
[835]	validation-rmse:6.29824
[836]	validation-rmse:6.29811
[837]	validation-rmse:6.29806
[838]	validation-rmse:6.29790
[839]	validation-rmse:6.29805
[840]	validation-rmse:6.29779
[841]	validation-rmse:6.29751
[842]	validation-rmse:6.29739
[843]	validation-rmse:6.29744
[844]	validation-rmse:6.29744
[845]	validation-rmse:6.29745
[846]	validation-rmse:6.29729
[847]	validation-rmse:6.29702
[848]	validation-rmse:6.29703
[849]	validation-rmse:6.29707
[850]	validation-rmse:6.29618
[851]	validation-rmse:6.29608
[852]	validation-rmse:6.29598
[853]	validation-rmse:6.29596
[854]	vali

2023/05/24 16:49:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


## Load the model and use it

In [20]:
logged_model = 'runs:/f9981d4e93484011b1ef5a0243b8f2f0/models_mlflow'

In [21]:
    # Load model as a xgboost.
    xgb_model = mlflow.xgboost.load_model(logged_model)

    # Prediction
    y_valid = xgb_model.predict(valid) # using DMatrix-typed validation data

[16:56:30] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
y_valid

array([15.072298 ,  7.2601957, 13.425392 , ..., 13.6989765,  6.6169147,
        8.195763 ], dtype=float32)

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
